In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [10]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'hole_diameter_sae'

In [11]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [15]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['HoleDiameter']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
# df = pd.concat([dfs.drop('custom_fields', axis=1)], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

4467


In [20]:
df['HoleDiameter'].explode().value_counts()

0.26"    95
0.38"    90
0.61"    90
0.17"    73
0.76"    64
0.56"    59
0.16"    58
0.81"    57
0.25"    51
0.36"    48
0.24"    36
0.48"    28
0.50"    28
0.34"    26
0.99"    25
0.39"    22
1.02"    21
0.15"    19
0.55"    19
0.45"    18
0.33"    16
0.18"    15
0.37"    15
0.46"    14
1.01"    14
0.31"    13
0.58"    13
0.28"    12
0.12"    11
0.82"    10
0.21"    10
0.27"    10
1.05"     8
0.77"     7
1.22"     7
1.26"     7
0.23"     6
0.22"     6
0.78"     6
1.66"     5
0.75"     5
1.25"     5
1.75"     5
0.49"     4
0.13"     4
0.42"     4
0.29"     3
1.56"     3
0.51"     3
0.59"     3
0.52"     2
0.95"     2
0.30"     2
1.62"     2
1.69"     2
0.67"     2
0.83"     2
0.35"     1
0.47"     1
1.04"     1
0.84"     1
1.06"     1
0.10"     1
0.60"     1
0.44"     1
0.80"     1
0.20"     1
0.41"     1
0.71"     1
1.19"     1
0.32"     1
0.72"     1
Name: HoleDiameter, dtype: int64

In [19]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [27]:
pattern=r'''(?i)(\d+\.?\d*)|()''' 
df['name'] = df['product_name'].apply(lambda x: re_extract(pattern, str(x)))
# df['ld'] = df['long_desc'].apply(lambda x: re_extract(pattern, str(x)))


pat=r'(?i)(\d+\.?\d*")|()'
df['matches']=df['HoleDiameter'].apply(lambda x: re_extract(pat, str(x)))
custom=df[df['matches'].astype(str)!='[]']
custom[curation_col]=custom['matches'].apply(lambda x: re.sub(r'.+to\s?|','',str(x))).apply(lambda x: re.sub(r"'|\[|\]",'',str(x)))
print(len(custom))
matchcustom=custom[['external_id',curation_col]]

1200


In [35]:
two=df[(df['matches'].astype(str)=='[]')]
# two['name']=two['name'].apply(lambda x: re.sub(r'(?i)\(|x','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'\s?\,\s?',',',str(x))).apply(lambda x: re.sub(r'"\]','""]',str(x))).apply(lambda x:re.sub(r'\[""|\["','["',str(x)))                                                                  
pat=r'(?i)((?i).{0,20}dia.{0,20})|()'
two['name']=two['product_name'].apply(lambda x: re_extract(pat, str(x)))
two['long']=two['long_desc'].apply(lambda x: re_extract(pat, str(x)))
two[(two['name'].astype(str)!='[]')|(two['long'].astype(str)!='[]')]

buckets external_id  \
61    Compressors Accessories & Replacement Parts      405911   
75    Compressors Accessories & Replacement Parts      541011   
76    Compressors Accessories & Replacement Parts      541005   
77    Compressors Accessories & Replacement Parts      541010   
78    Compressors Accessories & Replacement Parts      541032   
79    Compressors Accessories & Replacement Parts      541033   
80    Compressors Accessories & Replacement Parts      541031   
81    Compressors Accessories & Replacement Parts      541003   
82    Compressors Accessories & Replacement Parts      541004   
83    Compressors Accessories & Replacement Parts      541026   
84    Compressors Accessories & Replacement Parts      541007   
85    Compressors Accessories & Replacement Parts      541028   
86    Compressors Accessories & Replacement Parts      541029   
87    Compressors Accessories & Replacement Parts      541008   
88    Compressors Accessories & Replacement Parts      541009   
89    Compressors Accessories & Replacement Parts      541006   
90    Compressors Accessories & Replacement Parts      541025   
91    Compressors Accessories & Replacement Parts      541024   
92    Compressors Accessories & Replacement Parts      541023   
93    Compressors Accessories & Replacement Parts      541027   
94    Compressors Accessories & Replacement Parts      541030   
1773  Compressors Accessories & Replacement Parts   FF91064X4   
1774  Compressors Accessories & Replacement Parts   FF91064X6   
1775  Compressors Accessories & Replacement Parts   FF91064X8   
3626  Compressors Accessories & Replacement Parts       90943   
3627  Compressors Accessories & Replacement Parts       90944   
3628  Compressors Accessories & Replacement Parts       90941   
3629  Compressors Accessories & Replacement Parts       90942   
3630  Compressors Accessories & Replacement Parts       90940   
3725  Compressors Accessories & Replacement Parts       50395   
3726  Compressors Accessories & Replacement Parts       51966   
4466  Compressors Accessories & Replacement Parts     509005U   

                                              product_name  \
61                                 Service Plug Assortment   
75                                       1/4 In-Line Valve   
76                                         1" Flare Nozzle   
77                                       1/4 Swivel Nozzle   
78                                     2-1/2 Swivel Nozzle   
79                                       1/2 In-Line Valve   
80                                       1/2 Elbow Fitting   
81                                        1/8 Round Nozzle   
82                                        1/4 Round Nozzle   
83                                      1-1/4 Flare Nozzle   
84                                       1/4 NPT Connector   
85                                       3/8 NPT Connector   
86                                       1/2 NPT Connector   
87                                           1/4 Y-Fitting   
88                                       1/4 Elbow Fitting   
89                                       1/8 NPT Connector   
90                                        1/2 Round Nozzle   
91                                        3/8 Round Nozzle   
92                      1/4 Round Nozzle (1/2 Hose System)   
93                                      2-1/2 Flare Nozzle   
94                                           1/2 Y-Fitting   
1773                   1/4 Synflex 'Z' Series Crimp Sleeve   
1774                   3/8 Synflex 'Z' Series Crimp Sleeve   
1775                   1/2 Synflex 'Z' Series Crimp Sleeve   
3626                               Service Plug Assortment   
3627                                      Service Plug Kit   
3628                        3/16 - 7/8 Medium Service Plug   
3629                        1/2 - 1-3/4 Large Service Plug   
3630                         3/64 - 3/8 Small Service Plug   
3725                         Buna-N 

# N/A

In [37]:
na=two[(two['name'].astype(str)=='[]')|(two['long'].astype(str)=='[]')]
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
print(len(na)+len(matchcustom))

4467


In [38]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcustom))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))


# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

matchcustom
matchna


In [11]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [39]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcustom) 

In [40]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 